In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from time import time
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn import metrics


In [2]:
df  = pd.read_csv(r'/home/kekeing/Desktop/code/DateMining/data/lianjia_processed.csv',sep=',')
df.head()

,address,house_age,deal_time,deal_totalPrice,deal_unitPrice,floor_number,gross_area,house_decoration,house_orientation,house_usage,household_style,is_two_five,longitude_latitude,region,with_elevator,xiaoqu,lon,lat
0,qianguan,11,2016-07,53,7391,10,71.71,其他,南,普通住宅,2室1厅1厨1卫,0,"121.657898,39.025583",ganjingzi,1,盐岛湖,121.657898,39.025583
1,yishanli,13,2016-01,27,5639,15,47.00,其他,南,普通住宅,1室1厅1厨1卫,0,"121.80001712729,39.063714218257",kaifaqudl,1,新东方公寓二期,121.800017,39.063714
2,zhanqian,10,2016-01,43,6344,18,67.00,精装,南,普通住宅,2室1厅1厨1卫,0,"121.72275181639,39.092176577691",jinzhou,1,金连福地,121.722752,39.092177
3,lvshunkou1,6,2018-06,68,8418,29,80.67,毛坯,南,普通住宅,2室1厅1厨1卫,0,"121.337111,38.8275",lvshunkou,1,亿达蓝山,121.337111,38.827500
4,lvshunkou1,10,2019-09,44,5117,18,86.00,其他,南 北,普通住宅,2室2厅1厨1卫,0,"121.15637357126,38.781604479132",lvshunkou,1,华通和平海岸二期,121.156374,38.781604


In [3]:
df  = df[['deal_totalPrice','gross_area']]
da = df.to_numpy()
labels = da[:,-1]

In [4]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,metric='euclidean'),
             )
            )

In [5]:
print('init\t\ttime\tinertia\t\thomo\tcompl\tv-meas\tAMI\tsilhouette')
for n in range(2,5):
    bench_k_means(KMeans(init='k-means++', n_clusters=n, n_init=20),
              name="k-means++", data=da)
for n in range(2,5):
    bench_k_means(KMeans(init='random', n_clusters=n, n_init=20),
              name="random", data=da)

init		time	inertia		homo	compl	v-meas	AMI	silhouette
k-means++	1.82s	170209374	0.030	0.622	0.057	0.040	0.632
k-means++	3.18s	106797320	0.051	0.578	0.094	0.065	0.518
k-means++	4.26s	79229808	0.069	0.595	0.124	0.086	0.458
random   	1.44s	170209801	0.030	0.622	0.057	0.040	0.632
random   	2.53s	106798017	0.051	0.578	0.094	0.064	0.518
random   	4.28s	79236992	0.068	0.594	0.122	0.084	0.462


In [6]:
print('init\t\ttime\tinertia\t\thomo\tcompl\tv-meas\tAMI\tsilhouette')
for n in range(2,5):
    bench_k_means(MiniBatchKMeans(init='k-means++', n_clusters=n, n_init=20,batch_size=300),name="k-means", data=da)

init		time	inertia		homo	compl	v-meas	AMI	silhouette
k-means  	0.34s	171016248	0.032	0.598	0.061	0.043	0.613
k-means  	0.39s	106991535	0.053	0.581	0.097	0.067	0.510
k-means  	0.40s	79765164	0.066	0.594	0.119	0.083	0.468
